# BIQL Tutorial Guide

Welcome to the BIQL (BIDS Query Language) tutorial! This guide will walk you through
using BIQL to query BIDS neuroimaging datasets. We'll start with basic queries and
progressively explore more advanced features.

## What is BIQL?

BIQL is a SQL-like query language designed specifically for querying Brain Imaging 
Data Structure (BIDS) datasets. It allows you to:

- Search for specific files based on BIDS entities (subject, session, task, etc.)
- Filter data using metadata from JSON sidecars
- Access participant information from participants.tsv
- Perform aggregations and grouping operations
- Export results in various formats

## Prerequisites

First, let's set up our environment and get the example data:

In [1]:
import tempfile
from pathlib import Path
from biql import create_query_engine
import sys

# Install BIQL if running in Colab
if 'google.colab' in sys.modules:
    !pip install git+https://github.com/astewartau/biql.git > /dev/null 2>&1

# Set up paths - use a temporary directory that works in different environments
bids_examples_dir = Path(tempfile.gettempdir()) / "bids-examples"

# Clone bids-examples if it doesn't exist
if not bids_examples_dir.exists():
    !git clone https://github.com/bids-standard/bids-examples.git {bids_examples_dir} > /dev/null 2>&1

## Part 1: Basic Queries

Let's start with the synthetic dataset from bids-examples. This is a simple dataset
that's perfect for learning BIQL basics.

In [2]:
dataset_path = bids_examples_dir / "synthetic"
q = create_query_engine(dataset_path)
q.dataset_stats()

{'total_files': 60,
 'total_subjects': 5,
 'files_by_datatype': {'anat': 10, 'func': 30, 'beh': 5},
 'subjects': ['01', '02', '03', '04', '05'],
 'datatypes': ['anat', 'beh', 'func']}

### Simple Entity Queries

The most basic BIQL queries filter files by BIDS entities. You can query by any
BIDS entity that appears in your filenames:

In [3]:
q.run_query("sub=01", format="dataframe").head(5)

,filepath,relative_path,filename,sub,ses,suffix,datatype,extension,metadata,participants,task,run
0,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...,sub-01/ses-02/anat/sub-01_ses-02_T1w.nii,sub-01_ses-02_T1w.nii,01,02,T1w,anat,.nii,{},age=34; sex=F,NaN,NaN
1,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-02_bold.nii,01,02,bold,func,.nii,{},age=34; sex=F,nback,02
2,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-01_bold.nii,01,02,bold,func,.nii,{},age=34; sex=F,nback,01
3,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-rest_bol...,sub-01_ses-02_task-rest_bold.nii,01,02,bold,func,.nii,{},age=34; sex=F,rest,NaN
4,/tmp/bids-examples/synthetic/sub-01/ses-01/ana...,sub-01/ses-01/anat/sub-01_ses-01_T1w.nii,sub-01_ses-01_T1w.nii,01,01,T1w,anat,.nii,{},age=34; sex=F,NaN,NaN


In [4]:
results = q.run_query("datatype=func")
len(results)  # Number of functional files

30

In [5]:
q.run_query("SELECT DISTINCT task WHERE datatype=func", format="dataframe")

,task,_file_paths
0,nback,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
1,rest,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...


### Combining Conditions

You can combine multiple conditions using AND, OR, and NOT operators:

In [6]:
q.run_query("datatype=anat AND suffix=T1w", format="dataframe").head(5)

,filepath,relative_path,filename,sub,ses,suffix,datatype,extension,metadata,participants
0,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...,sub-01/ses-02/anat/sub-01_ses-02_T1w.nii,sub-01_ses-02_T1w.nii,01,02,T1w,anat,.nii,{},age=34; sex=F
1,/tmp/bids-examples/synthetic/sub-01/ses-01/ana...,sub-01/ses-01/anat/sub-01_ses-01_T1w.nii,sub-01_ses-01_T1w.nii,01,01,T1w,anat,.nii,{},age=34; sex=F
2,/tmp/bids-examples/synthetic/sub-04/ses-02/ana...,sub-04/ses-02/anat/sub-04_ses-02_T1w.nii,sub-04_ses-02_T1w.nii,04,02,T1w,anat,.nii,{},age=21; sex=F
3,/tmp/bids-examples/synthetic/sub-04/ses-01/ana...,sub-04/ses-01/anat/sub-04_ses-01_T1w.nii,sub-04_ses-01_T1w.nii,04,01,T1w,anat,.nii,{},age=21; sex=F
4,/tmp/bids-examples/synthetic/sub-05/ses-02/ana...,sub-05/ses-02/anat/sub-05_ses-02_T1w.nii,sub-05_ses-02_T1w.nii,05,02,T1w,anat,.nii,{},age=42; sex=M


In [7]:
q.run_query("task=nback OR task=rest", format="dataframe").head(5)

,filepath,relative_path,filename,sub,ses,task,run,suffix,datatype,extension,metadata,participants
0,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-02_bold.nii,01,02,nback,02,bold,func,.nii,{},age=34; sex=F
1,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-01_bold.nii,01,02,nback,01,bold,func,.nii,{},age=34; sex=F
2,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-rest_bol...,sub-01_ses-02_task-rest_bold.nii,01,02,rest,NaN,bold,func,.nii,{},age=34; sex=F
3,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...,sub-01/ses-01/func/sub-01_ses-01_task-nback_ru...,sub-01_ses-01_task-nback_run-02_bold.nii,01,01,nback,02,bold,func,.nii,{},age=34; sex=F
4,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...,sub-01/ses-01/func/sub-01_ses-01_task-rest_bol...,sub-01_ses-01_task-rest_bold.nii,01,01,rest,NaN,bold,func,.nii,{},age=34; sex=F


### Using WHERE Clause

For more SQL-like queries, you can use the WHERE clause:

In [8]:
q.run_query("WHERE sub=01 AND datatype=func", format="dataframe")

,filepath,relative_path,filename,sub,ses,task,run,suffix,datatype,extension,metadata,participants
0,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-02_bold.nii,01,02,nback,02,bold,func,.nii,{},age=34; sex=F
1,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-nback_ru...,sub-01_ses-02_task-nback_run-01_bold.nii,01,02,nback,01,bold,func,.nii,{},age=34; sex=F
2,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...,sub-01/ses-02/func/sub-01_ses-02_task-rest_bol...,sub-01_ses-02_task-rest_bold.nii,01,02,rest,NaN,bold,func,.nii,{},age=34; sex=F
3,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...,sub-01/ses-01/func/sub-01_ses-01_task-nback_ru...,sub-01_ses-01_task-nback_run-02_bold.nii,01,01,nback,02,bold,func,.nii,{},age=34; sex=F
4,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...,sub-01/ses-01/func/sub-01_ses-01_task-rest_bol...,sub-01_ses-01_task-rest_bold.nii,01,01,rest,NaN,bold,func,.nii,{},age=34; sex=F
5,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...,sub-01/ses-01/func/sub-01_ses-01_task-nback_ru...,sub-01_ses-01_task-nback_run-01_bold.nii,01,01,nback,01,bold,func,.nii,{},age=34; sex=F


## Part 2: SELECT Clause and Field Selection

By default, BIQL returns all available fields. Use SELECT to choose specific fields:

In [ ]:
q.run_query(
    "SELECT sub, task, run, filename WHERE datatype=func",
    format="dataframe"
).head(5)

,sub,task,run,filename,_file_paths
0,01,nback,02,sub-01_ses-02_task-nback_run-02_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
1,01,nback,01,sub-01_ses-02_task-nback_run-01_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
2,01,rest,None,sub-01_ses-02_task-rest_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
3,01,nback,02,sub-01_ses-01_task-nback_run-02_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
4,01,rest,None,sub-01_ses-01_task-rest_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
5,01,nback,01,sub-01_ses-01_task-nback_run-01_bold.nii,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
6,04,nback,02,sub-04_ses-02_task-nback_run-02_bold.nii,/tmp/bids-examples/synthetic/sub-04/ses-02/fun...
7,04,nback,01,sub-04_ses-02_task-nback_run-01_bold.nii,/tmp/bids-examples/synthetic/sub-04/ses-02/fun...
8,04,rest,None,sub-04_ses-02_task-rest_bold.nii,/tmp/bids-examples/synthetic/sub-04/ses-02/fun...
9,04,nback,02,sub-04_ses-01_task-nback_run-02_bold.nii,/tmp/bids-examples/synthetic/sub-04/ses-01/fun...


In [10]:
q.run_query(
    "SELECT sub, relative_path WHERE suffix=T1w",
    format="dataframe"
)

,sub,relative_path,_file_paths
0,01,sub-01/ses-02/anat/sub-01_ses-02_T1w.nii,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...
1,01,sub-01/ses-01/anat/sub-01_ses-01_T1w.nii,/tmp/bids-examples/synthetic/sub-01/ses-01/ana...
2,04,sub-04/ses-02/anat/sub-04_ses-02_T1w.nii,/tmp/bids-examples/synthetic/sub-04/ses-02/ana...
3,04,sub-04/ses-01/anat/sub-04_ses-01_T1w.nii,/tmp/bids-examples/synthetic/sub-04/ses-01/ana...
4,05,sub-05/ses-02/anat/sub-05_ses-02_T1w.nii,/tmp/bids-examples/synthetic/sub-05/ses-02/ana...
5,05,sub-05/ses-01/anat/sub-05_ses-01_T1w.nii,/tmp/bids-examples/synthetic/sub-05/ses-01/ana...
6,02,sub-02/ses-02/anat/sub-02_ses-02_T1w.nii,/tmp/bids-examples/synthetic/sub-02/ses-02/ana...
7,02,sub-02/ses-01/anat/sub-02_ses-01_T1w.nii,/tmp/bids-examples/synthetic/sub-02/ses-01/ana...
8,03,sub-03/ses-02/anat/sub-03_ses-02_T1w.nii,/tmp/bids-examples/synthetic/sub-03/ses-02/ana...
9,03,sub-03/ses-01/anat/sub-03_ses-01_T1w.nii,/tmp/bids-examples/synthetic/sub-03/ses-01/ana...


## Part 3: Pattern Matching

BIQL supports wildcards and regular expressions for flexible matching:

In [11]:
results = q.run_query("suffix=*bold*")
len(results)  # Count of files with 'bold' in suffix

30

In [12]:
q.run_query(
    "SELECT DISTINCT task WHERE task~=\".*back.*\"",
    format="dataframe"
)

,task,_file_paths
0,nback,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...


## Part 4: Ranges and Lists

BIQL supports range queries and IN operators for matching multiple values:

### DISTINCT vs Non-DISTINCT Aggregations

BIQL supports both DISTINCT and non-DISTINCT array aggregations:

- **With DISTINCT**: `ARRAY_AGG(DISTINCT field)` returns only unique non-null values
- **Without DISTINCT**: `ARRAY_AGG(field)` returns all values including duplicates and nulls

The count of items in a non-DISTINCT array will match `COUNT(*)` for the group:

In [13]:
q.run_query(
    "SELECT sub, ARRAY_AGG(DISTINCT task) as tasks, COUNT(*) as total_files "
    "WHERE sub IN ['01', '02', '03'] "
    "GROUP BY sub",
    format="json"
)

[{'sub': '01',
  'tasks': ['nback', 'rest', 'stroop'],
  'total_files': 12,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/sub-01_sessions.tsv',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/sub-01_ses-02_scans.tsv',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/sub-01_ses-01_scans.t

In [14]:
q.run_query(
    "SELECT task, run, COUNT(*) as file_count, "
    "COUNT(DISTINCT sub) as subjects "
    "WHERE datatype=func "
    "GROUP BY task, run "
    "ORDER BY task, run",
    format="dataframe"
)

,task,run,file_count,subjects,_file_paths
0,nback,01,10,5,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
1,nback,02,10,5,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
2,rest,None,10,5,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...


## Part 5: Grouping and Aggregation

BIQL supports SQL-like grouping and aggregation functions:

In [15]:
q.run_query("SELECT sub, COUNT(*) GROUP BY sub", format="dataframe")

,sub,count,_file_paths
0,01,12,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...
1,04,12,/tmp/bids-examples/synthetic/sub-04/ses-02/ana...
2,05,12,/tmp/bids-examples/synthetic/sub-05/ses-02/ana...
3,02,12,/tmp/bids-examples/synthetic/sub-02/ses-02/ana...
4,03,12,/tmp/bids-examples/synthetic/sub-03/ses-02/ana...


In [16]:
q.run_query(
    "SELECT sub, datatype, COUNT(*) GROUP BY sub, datatype",
    format="json"
)

[{'sub': '01',
  'datatype': 'anat',
  'count': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii']},
 {'sub': '01',
  'datatype': 'func',
  'count': 6,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_bold.nii']},
 {'sub': '04',
  'datatype': 'anat',
  'count': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-04/ses-02/anat/sub-04_ses-02_T1w.nii',
   '/tmp/bids-ex

## Part 6: Working with Metadata

BIQL can query JSON sidecar metadata using the `metadata.` namespace.
The synthetic dataset has task-level metadata files like `task-nback_bold.json`:

In [17]:
q.run_query(
    "SELECT task, COUNT(*) as file_count, "
    "ARRAY_AGG(DISTINCT sub) as subjects_with_task, "
    "ARRAY_AGG(DISTINCT datatype) as datatypes "
    "GROUP BY task",
    format="json"
)

[{'task': None,
  'file_count': 25,
  'subjects_with_task': ['01', '02', '03', '04', '05'],
  'datatypes': ['anat'],
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-04/ses-02/anat/sub-04_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-04/ses-01/anat/sub-04_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-05/ses-02/anat/sub-05_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-05/ses-01/anat/sub-05_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-02/ses-02/anat/sub-02_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-02/ses-01/anat/sub-02_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-03/ses-02/anat/sub-03_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-03/ses-01/anat/sub-03_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/sub-01_sessions.tsv',
   '/tmp/bids-examples/synthetic/sub-01/ses-02

In [18]:
q.run_query(
    "SELECT datatype, COUNT(*) as total_files, "
    "COUNT(DISTINCT sub) as subjects, "
    "ARRAY_AGG(DISTINCT sub) as subject_list "
    "GROUP BY datatype "
    "ORDER BY total_files DESC",
    format="json"
)

[{'datatype': 'anat',
  'total_files': 10,
  'subjects': 5,
  'subject_list': ['01', '02', '03', '04', '05'],
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-04/ses-02/anat/sub-04_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-04/ses-01/anat/sub-04_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-05/ses-02/anat/sub-05_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-05/ses-01/anat/sub-05_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-02/ses-02/anat/sub-02_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-02/ses-01/anat/sub-02_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-03/ses-02/anat/sub-03_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-03/ses-01/anat/sub-03_ses-01_T1w.nii']},
 {'datatype': 'func',
  'total_files': 30,
  'subjects': 5,
  'subject_list': ['01', '02', '03', '04', '05'],
  '

## Part 7: Participant Information

Access participant demographics using the `participants.` namespace:

In [19]:
q.run_query(
    "SELECT DISTINCT sub, participants.age, participants.sex",
    format="dataframe"
)

,sub,participants.age,participants.sex,_file_paths
0,01,34,F,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...
1,04,21,F,/tmp/bids-examples/synthetic/sub-04/ses-02/ana...
2,05,42,M,/tmp/bids-examples/synthetic/sub-05/ses-02/ana...
3,02,38,M,/tmp/bids-examples/synthetic/sub-02/ses-02/ana...
4,03,22,M,/tmp/bids-examples/synthetic/sub-03/ses-02/ana...


In [20]:
q.run_query(
    "SELECT sub, task, participants.age WHERE participants.age > 25",
    format="dataframe"
)

,sub,task,participants.age,_file_paths
0,01,None,34,/tmp/bids-examples/synthetic/sub-01/ses-02/ana...
1,01,nback,34,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
2,01,nback,34,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
3,01,rest,34,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
4,01,None,34,/tmp/bids-examples/synthetic/sub-01/ses-01/ana...
5,01,nback,34,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
6,01,rest,34,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
7,01,nback,34,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
8,05,None,42,/tmp/bids-examples/synthetic/sub-05/ses-02/ana...
9,05,rest,42,/tmp/bids-examples/synthetic/sub-05/ses-02/fun...


## Part 8: Advanced Queries

Let's combine multiple features for more complex queries:

In [21]:
q.run_query("""
    SELECT sub, ses, task, COUNT(*) as n_runs
    WHERE datatype=func AND task != rest
    GROUP BY sub, ses, task
    HAVING COUNT(*) > 1
    ORDER BY sub, task
""", format="json")

[{'sub': '01',
  'ses': '02',
  'task': 'nback',
  'n_runs': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii']},
 {'sub': '01',
  'ses': '01',
  'task': 'nback',
  'n_runs': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_bold.nii']},
 {'sub': '02',
  'ses': '02',
  'task': 'nback',
  'n_runs': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-02/ses-02/func/sub-02_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-02/ses-02/func/sub-02_ses-02_task-nback_run-01_bold.nii']},
 {'sub': '02',
  'ses': '01',
  'task': 'nback',
  'n_runs': 2,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-02/ses-01/func/sub-02_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids

In [22]:
q.run_query("""
    SELECT sub, task,
           ARRAY_AGG(filename WHERE suffix='bold') as imaging_files,
           ARRAY_AGG(filename WHERE run='01') as run01_files,
           ARRAY_AGG(filename WHERE run='02') as run02_files
    WHERE datatype=func
    GROUP BY sub, task
""", format="table")  # Using table format since arrays don't display well in dataframes

'| imaging_files   | run01_files     | run02_files     | sub | task  |\n| --------------- | --------------- | --------------- | --- | ----- |\n| [...4 items...] | [...2 items...] | [...2 items...] | 01  | nback |\n| [...2 items...] | [...0 items...] | [...0 items...] | 01  | rest  |\n| [...4 items...] | [...2 items...] | [...2 items...] | 04  | nback |\n| [...2 items...] | [...0 items...] | [...0 items...] | 04  | rest  |\n| [...2 items...] | [...0 items...] | [...0 items...] | 05  | rest  |\n| [...4 items...] | [...2 items...] | [...2 items...] | 05  | nback |\n| [...2 items...] | [...0 items...] | [...0 items...] | 02  | rest  |\n| [...4 items...] | [...2 items...] | [...2 items...] | 02  | nback |\n| [...4 items...] | [...2 items...] | [...2 items...] | 03  | nback |\n| [...2 items...] | [...0 items...] | [...0 items...] | 03  | rest  |'

## Part 9: Output Formats

BIQL supports multiple output formats for different use cases:

In [23]:
sample_query = "SELECT sub, task, run WHERE datatype=func AND sub=01"

print(q.run_query(sample_query, format="table"))

| run | sub | task  |
| --- | --- | ----- |
| 02  | 01  | nback |
| 01  | 01  | nback |
|     | 01  | rest  |
| 02  | 01  | nback |
|     | 01  | rest  |
| 01  | 01  | nback |


In [24]:
print(q.run_query(sample_query, format="csv"))

run,sub,task
02,01,nback
01,01,nback
,01,rest
02,01,nback
,01,rest
01,01,nback



In [25]:
results_json = q.run_query(sample_query, format="json")
results_json[:2]  # Show first 2 entries

[{'sub': '01',
  'task': 'nback',
  'run': '02',
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii']},
 {'sub': '01',
  'task': 'nback',
  'run': '01',
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii']}]

In [26]:
print(q.run_query("WHERE sub=01 AND suffix=T1w", format="paths"))

/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii
/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii


In [27]:
q.run_query(sample_query, format="dataframe")

,sub,task,run,_file_paths
0,01,nback,02,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
1,01,nback,01,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
2,01,rest,None,/tmp/bids-examples/synthetic/sub-01/ses-02/fun...
3,01,nback,02,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
4,01,rest,None,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...
5,01,nback,01,/tmp/bids-examples/synthetic/sub-01/ses-01/fun...


## Part 10: Real-World Examples

Let's look at some practical queries you might use in neuroimaging research:

In [28]:
q.run_query("""
    SELECT sub, 
           COUNT(*) as total_files,
           COUNT(DISTINCT datatype) as datatypes,
           ARRAY_AGG(DISTINCT datatype) as available_data
    GROUP BY sub
""", format="json")

[{'sub': '01',
  'total_files': 12,
  'datatypes': 3,
  'available_data': ['anat', 'beh', 'func'],
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/sub-01_sessions.tsv',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/sub-01_ses-02_scans.tsv',
   '/tmp/bids-examples/synthetic/sub-01/ses-0

In [29]:
q.run_query("""
    SELECT sub, ses,
           COUNT(*) as files_per_session,
           ARRAY_AGG(DISTINCT task) as tasks_in_session
    GROUP BY sub, ses
""", format="json")

[{'sub': '01',
  'ses': '02',
  'files_per_session': 5,
  'tasks_in_session': ['nback', 'rest'],
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/anat/sub-01_ses-02_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/sub-01_ses-02_scans.tsv']},
 {'sub': '01',
  'ses': '01',
  'files_per_session': 6,
  'tasks_in_session': ['nback', 'rest', 'stroop'],
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-01/anat/sub-01_ses-01_T1w.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run

In [30]:
q.run_query("""
    SELECT sub,
           COUNT(DISTINCT task) as unique_tasks,
           ARRAY_AGG(DISTINCT task) as completed_tasks,
           COUNT(*) as total_functional_files
    WHERE datatype=func
    GROUP BY sub
    HAVING COUNT(DISTINCT task) > 1  # Subjects with multiple tasks
""", format="json")

[{'sub': '01',
  'unique_tasks': 2,
  'completed_tasks': ['nback', 'rest'],
  'total_functional_files': 6,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-02/func/sub-01_ses-02_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-rest_bold.nii',
   '/tmp/bids-examples/synthetic/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_bold.nii']},
 {'sub': '04',
  'unique_tasks': 2,
  'completed_tasks': ['nback', 'rest'],
  'total_functional_files': 6,
  '_file_paths': ['/tmp/bids-examples/synthetic/sub-04/ses-02/func/sub-04_ses-02_task-nback_run-02_bold.nii',
   '/tmp/bids-examples/synthetic/sub-04/ses-02/func/sub-04_ses-02_task-nback_run-01_bold.nii',
   '/tmp/b

## Summary

You've learned how to:

1. **Basic queries**: Filter by BIDS entities
2. **Logical operators**: Combine conditions with AND, OR, NOT
3. **SELECT clause**: Choose specific fields to return
4. **Pattern matching**: Use wildcards and regex
5. **Ranges and lists**: Query multiple values efficiently
6. **Aggregations**: Count and group data
7. **Metadata queries**: Access JSON sidecar information
8. **Participant data**: Query demographics
9. **Complex queries**: Combine multiple features
10. **Output formats**: Export results in different formats

## Next Steps

- Check out the [Language Reference](language.md) for complete syntax details
- Explore more [examples](../examples/) for specific use cases
- Use the CLI tool `biql` for command-line queries
- Integrate BIQL into your Python analysis pipelines

Happy querying!